In [1]:
# !pip install parse
# !pip install matplotlib --upgrade

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

colab_datadir = '/content/drive/MyDrive/sharedCodeBase/FamilyAnalysis/'
local_datadir = './'
datadir = local_datadir
import sys
sys.path.append(datadir)

In [3]:
import pandas as pd
import numpy as np

from parse import *
import os
import sklearn
from sklearn.datasets import make_blobs
from matplotlib import pyplot as plt
#%matplotlib auto
import matplotlib.image as mpimg 
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.collections import PolyCollection
from collections import defaultdict
import datetime
from datetime import timedelta
import argparse
from myHelper import *
from graphHelper import *

# 选择家庭



In [4]:
class Myclass():
    MinCountOfPoints=6 #这里我没改，因为分长时间和短时间计算时，我手动一起调了（详见最后一个part）
    MinCountOfPointsForAll=-1
    SizeOfGrid='1' # 不需要调 
    figure=datadir+'floorplan01.png' # 默认
    path=datadir+'results_both/' # 默认
    res=datadir+'figures_both/' # 默认
    bySecond=False
    IntervalOfMinutesOrSeconds=1 # 1分钟 1个点  
    setConfig=True #False

arguments=Myclass()

##########################################
# Choose Family
##########################################
family_config = chooseFamily()

max_x = family_config.size[1]
min_x = family_config.size[0]
max_y = family_config.size[3]
min_y = family_config.size[2]

#需要改（mytask和advanced都需要改）
if family_config.name == "ParentsOfSun":
  arguments.figure = datadir+'floorplan01.png'
  arguments.path = datadir+'family1/results_tag1/'
  arguments.res=datadir+'family1/figures_tag1/'
elif family_config.name == "UncleOfSun":
  arguments.figure = datadir+'floorplan02.png'
  arguments.path = datadir+'family2/results_tag0/'
  arguments.res=datadir+'family2/figures_tag0/'
elif family_config.name == "AuntOfSun":
  arguments.figure = datadir+'floorplan03.png'
  arguments.path = datadir+'family3/results_tag2/'
  arguments.res=datadir+'family3/figures_tag2/'
else:
  print("Family doesn't exist !\n")
  exit(0)

unit = float(arguments.SizeOfGrid)  #网格单元长宽m
column_num = int((max_x-min_x)//unit)
row_num = int((max_y-min_y)//unit)
map_img = mpimg.imread(arguments.figure)

# time epsilon to select the data point
epsilon = 3 #timedelta(minutes=10)
min_count = arguments.MinCountOfPoints

# define time
year = 2022
input_path = arguments.path
res_path = arguments.res
C = 0

==>Familiy choices :<==
1 :ParentsOfSun
2 :UncleOfSun
3 :AuntOfSun
4 :Zeng
1
Family of ParentsOfSun is chosen!

Trace familes:
	ParentsOfSun
	size of house : [-1.6, 13.55, -2.5, 10.55]
		Scenario=Shift
			zone 1: OutDoor
				{ x : [-1000.0,-0.0],y : [-1000.0,1000.0] => Delete
			zone 2: OutCanteen
				{ x : [-0.375,1.07],y : [6.885,1000.0] => MoveToDown
			zone kitchen: InKitchen
				{ x : [1.35,3.5],y : [7.165,10.26] => MoveToLeftLinearly
			zone 3: OutKitchen
				{ x : [1.35,3.5],y : [10.26,1000.0] => MoveToDown
			zone 4: OutKitchen
				{ x : [3.5,1000.0],y : [10.26,1000.0] => MoveToLeftDown
			zone 5: OutCanteen
				{ x : [3.5,1000.0],y : [4.985,6.885] => MoveToLeft
			zone 6: OutBalcony
				{ x : [-0.375,6.59],y : [-1000.0,-2.2] => MoveToUp
			zone 7: OutBedRoom
				{ x : [3.5,1000.0],y : [4.03,7.165] => MoveToLeft
			zone 8: OutLivingroom
				{ x : [3.89,1000.0],y : [-0.36,4.03] => MoveToLeft


# Setup

## Read choosed data

Data Format (index,time,x,y,label,zone)

In [5]:
#这里需要改filename = "UncleOfSun_choice_data_all.xlsx"，每换一户需要改一次；同一户里换tag不需要改
filename = "ParentsOfSun_choice_data_all.xlsx" # 可以修改的输入数据路径
df = pd.read_excel(open(datadir+filename,'rb'))
# Drop first column of dataframe
df = df.iloc[: , 1:]
# Drop wrong labels (再次检查-1 label)
df = df.drop(df[df.label.isin([-1])].index)
df

,time,x,y,label,zone
0,2022-02-02 07:29:00,2.729071,5.433207,110,3
1,2022-02-02 07:30:00,2.009183,6.168583,124,3
2,2022-02-02 07:31:00,2.896620,7.432667,140,1
3,2022-02-02 07:32:00,2.875584,7.700000,155,1
4,2022-02-02 07:33:00,2.957817,7.512167,140,1
...,...,...,...,...,...
4593,2022-02-13 22:45:00,2.818089,9.115000,170,1
4594,2022-02-13 22:46:00,2.808154,9.260000,170,1
4595,2022-02-13 22:47:00,2.808154,9.260000,170,1
4596,2022-02-13 22:48:00,2.932453,8.690770,170,1


## Configuration

设置划分常住区域和非常住区域的百分比

In [6]:
#需要划分常驻和非常驻的百分比，意思是网格密度大于各区域最高网格密度 X% 的则是常驻
percentage = 0.2

In [7]:
df["label"].value_counts()

64     817
170    483
63     426
111    405
49     369
155    322
4      178
110    169
140    143
48     130
125    110
95     108
94     100
169     86
79      82
156     68
141     61
78      57
109     45
171     41
3       38
18      34
34      33
185     31
124     30
19      29
62      28
93      28
33      26
184     25
126     20
154     20
108     17
36      15
186     14
139     10
Name: label, dtype: int64

# Normalization ： 对每个区域的每个网格的点数进行归一化处理

## 对数据分组

In [8]:
df_groupby = df.groupby(by=["zone","label"])
df_count = df_groupby.count().rename(columns = {"x" : "number"})
df_count

time  number    y
zone label                   
0    33       22      22   22
     34       26      26   26
     36       15      15   15
     48      130     130  130
     49      369     369  369
     62       28      28   28
     63      426     426  426
     64      817     817  817
     78       57      57   57
     79       82      82   82
     93       22      22   22
     94        2       2    2
1    139       9       9    9
     140     117     117  117
     141      50      50   50
     154      20      20   20
     155     322     322  322
     156      68      68   68
     169      86      86   86
     170     483     483  483
     171      41      41   41
     184      25      25   25
     185      31      31   31
     186      14      14   14
2    3        38      38   38
     4       178     178  178
     18       34      34   34
     19       29      29   29
     33        4       4    4
     34        7       7    7
3    93        6       6    6
     94       98      98   98
     95      108     108  108
     108      17      17   17
     109      45      45   45
     110     169     169  169
     111     405     405  405
     124      30      30   30
     125     110     110  110
     126      20      20   20
     139       1       1    1
     140      26      26   26
     141      11      11   11

## 对于出现在不同区的相同label，我们进行整理，选择数量更多的zone保存该label

In [9]:
df_count_index = df_count.reset_index() # 重新整理表格
# 对处于不同zone的相同label进行处理
new_df_count = (df_count_index.sort_values("number",ascending=False)
                  #.drop_duplicates(subset=["label"])
                  .groupby(['label']).agg({"number":"sum","zone":"first"})
                  .reset_index())
# 按照降序打印
new_df_count.sort_values("number",ascending=False)

,label,number,zone
11,64,817,0
31,170,483,1
10,63,426,0
20,111,405,3
8,49,369,0
28,155,322,1
1,4,178,2
19,110,169,3
25,140,143,1
7,48,130,0


## 对数据进行归一化处理

In [10]:

# df_normalized = new_df_count.groupby("zone").apply(lambda x: x.number / x.number.max())

zone_list = new_df_count.zone.unique()
grouped = new_df_count.groupby('zone')
split_dfs = []
for zone in zone_list:
  current_group  = grouped.get_group(zone)
  current_group["number"] = current_group.number/current_group.number.max()
  split_dfs.append(current_group)
df_normalized = pd.concat(split_dfs)

<ipython-input-10-ed77763fbf0d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_group["number"] = current_group.number/current_group.number.max()


In [11]:
print(df_normalized)

    label    number  zone
0       3  0.213483     2
1       4  1.000000     2
2      18  0.191011     2
3      19  0.162921     2
4      33  0.031824     0
5      34  0.040392     0
6      36  0.018360     0
7      48  0.159119     0
8      49  0.451652     0
9      62  0.034272     0
10     63  0.521420     0
11     64  1.000000     0
12     78  0.069767     0
13     79  0.100367     0
14     93  0.034272     0
15     94  0.246914     3
16     95  0.266667     3
17    108  0.041975     3
18    109  0.111111     3
19    110  0.417284     3
20    111  1.000000     3
21    124  0.074074     3
22    125  0.271605     3
23    126  0.049383     3
24    139  0.020704     1
25    140  0.296066     1
26    141  0.126294     1
27    154  0.041408     1
28    155  0.666667     1
29    156  0.140787     1
30    169  0.178054     1
31    170  1.000000     1
32    171  0.084886     1
33    184  0.051760     1
34    185  0.064182     1
35    186  0.028986     1


## 添加其他信息进入表格

In [12]:
df_normalized["category"] = np.where(df_normalized['number'] > percentage, "常驻", "非常驻")
print(df_normalized)

    label    number  zone category
0       3  0.213483     2       常驻
1       4  1.000000     2       常驻
2      18  0.191011     2      非常驻
3      19  0.162921     2      非常驻
4      33  0.031824     0      非常驻
5      34  0.040392     0      非常驻
6      36  0.018360     0      非常驻
7      48  0.159119     0      非常驻
8      49  0.451652     0       常驻
9      62  0.034272     0      非常驻
10     63  0.521420     0       常驻
11     64  1.000000     0       常驻
12     78  0.069767     0      非常驻
13     79  0.100367     0      非常驻
14     93  0.034272     0      非常驻
15     94  0.246914     3       常驻
16     95  0.266667     3       常驻
17    108  0.041975     3      非常驻
18    109  0.111111     3      非常驻
19    110  0.417284     3       常驻
20    111  1.000000     3       常驻
21    124  0.074074     3      非常驻
22    125  0.271605     3       常驻
23    126  0.049383     3      非常驻
24    139  0.020704     1      非常驻
25    140  0.296066     1       常驻
26    141  0.126294     1      非常驻
27    154  0.041408 

## 统计常驻和非常驻面积

In [13]:
df_size = df_normalized.groupby(['zone','category']).count().reset_index().drop("label",axis=1)
df_size["size"] = df_size["number"]*unit**2
df_size

,zone,category,number,size
0,0,常驻,3,3.0
1,0,非常驻,8,8.0
2,1,常驻,3,3.0
3,1,非常驻,9,9.0
4,2,常驻,2,2.0
5,2,非常驻,2,2.0
6,3,常驻,5,5.0
7,3,非常驻,4,4.0


# 绘制常驻和非常驻区域

In [14]:
# Define colors 定义颜色
number_zones = len(df_normalized["zone"].unique())
#interval = 1/number_zones
interval = 1/6
dict_color = {}
for i in range(number_zones):
  dict_color["常驻"+str(i)] = (i+0.75) * interval
  dict_color["非常驻"+str(i)] = (i+0.25) * interval

dict_color

{'常驻0': 0.125,
 '非常驻0': 0.041666666666666664,
 '常驻1': 0.29166666666666663,
 '非常驻1': 0.20833333333333331,
 '常驻2': 0.4583333333333333,
 '非常驻2': 0.375,
 '常驻3': 0.625,
 '非常驻3': 0.5416666666666666}

In [15]:

# re-organize the grid
zone_grids = np.zeros([row_num,column_num])
grids = np.zeros([row_num,column_num])
# for i in df_normalized.label:
for index, row in df_normalized.iterrows():
    # parameters 
    i = int(row["label"])
    n = float(row["number"])
    z = int(row["zone"])
    # calculation
    r = (i-1)//column_num 
    c = i - column_num * r - 1 
    # row = df_normalized.loc[df_normalized.index[df_normalized["label"]==i]]
    if n>percentage:
      grids[r,c] = dict_color["常驻"+str(z)]   #0.8是颜色信息，深色
    else:
      grids[r,c] = dict_color["非常驻"+str(z)]  #0.2是颜色信息，浅色

    zone_grids[r,c] =  z

fig, ax = plt.subplots(figsize=(max_x-min_x,max_y-min_y))
ax = sns.heatmap(grids, cmap='Paired',  
                  linewidths=0,alpha=0.7, 
                  xticklabels =False,square = True,
                  annot=zone_grids,yticklabels =False,
                  mask=(grids==0.),center=0.5)   #cmap是颜色库；center=表示0.几是颜色区域中心点 #YlGnBu #rocket #coolwarm #cubehelix #Paired #rocket_r
ax.invert_yaxis()
#plt.show()
plt.imshow(map_img,zorder = 0, extent= [-0.1,(max_x-min_x)/unit-0.2,-0.1,(max_y-min_y)/unit-0.3])# [x_left/unit,x_right/unit,y_down/unit,y_up/unit])
parsed = parse("{}.{}",filename)
prefix = parsed[0]
fig.savefig(res_path+"analysis1_"+prefix+".png",dpi=300)
plt.close(fig)

fig, ax = plt.subplots(figsize=(max_x-min_x,max_y-min_y))
ax = sns.heatmap(grids, cmap='Paired',  
                  linewidths=0,alpha=0.7, 
                  xticklabels =False,square = True,
                  yticklabels =False,
                  mask=(grids==0.),center=0.5)   #cmap是颜色库；center=表示0.几是颜色区域中心点 #YlGnBu #rocket #coolwarm #cubehelix #Paired #rocket_r
ax.invert_yaxis()
#plt.show()
plt.imshow(map_img,zorder = 0, extent= [-0.1,(max_x-min_x)/unit-0.2,-0.1,(max_y-min_y)/unit-0.3])# [x_left/unit,x_right/unit,y_down/unit,y_up/unit])
parsed = parse("{}.{}",filename)
prefix = parsed[0]
fig.savefig(res_path+"analysis2_"+prefix+".png",dpi=300)
plt.close(fig)

# 绘制时间轴(需要配合TimeLineGantt程序使用)

## Re-formalization

In [16]:
# Original data
df

,time,x,y,label,zone
0,2022-02-02 07:29:00,2.729071,5.433207,110,3
1,2022-02-02 07:30:00,2.009183,6.168583,124,3
2,2022-02-02 07:31:00,2.896620,7.432667,140,1
3,2022-02-02 07:32:00,2.875584,7.700000,155,1
4,2022-02-02 07:33:00,2.957817,7.512167,140,1
...,...,...,...,...,...
4593,2022-02-13 22:45:00,2.818089,9.115000,170,1
4594,2022-02-13 22:46:00,2.808154,9.260000,170,1
4595,2022-02-13 22:47:00,2.808154,9.260000,170,1
4596,2022-02-13 22:48:00,2.932453,8.690770,170,1


In [17]:
# 归一化数据
df_normalized

,label,number,zone,category
0,3,0.213483,2,常驻
1,4,1.000000,2,常驻
2,18,0.191011,2,非常驻
3,19,0.162921,2,非常驻
4,33,0.031824,0,非常驻
5,34,0.040392,0,非常驻
6,36,0.018360,0,非常驻
7,48,0.159119,0,非常驻
8,49,0.451652,0,常驻
9,62,0.034272,0,非常驻


In [18]:
df_join = df.join(df_normalized.set_index('label'), on='label', lsuffix='_before', rsuffix='_after')
# 合成新的zone tag
df_join["zone_tag"] = df_join.apply(lambda row: str(row.zone_after) + row.category, axis=1)
df_join


,time,x,y,label,zone_before,number,zone_after,category,zone_tag
0,2022-02-02 07:29:00,2.729071,5.433207,110,3,0.417284,3,常驻,3常驻
1,2022-02-02 07:30:00,2.009183,6.168583,124,3,0.074074,3,非常驻,3非常驻
2,2022-02-02 07:31:00,2.896620,7.432667,140,1,0.296066,1,常驻,1常驻
3,2022-02-02 07:32:00,2.875584,7.700000,155,1,0.666667,1,常驻,1常驻
4,2022-02-02 07:33:00,2.957817,7.512167,140,1,0.296066,1,常驻,1常驻
...,...,...,...,...,...,...,...,...,...
4593,2022-02-13 22:45:00,2.818089,9.115000,170,1,1.000000,1,常驻,1常驻
4594,2022-02-13 22:46:00,2.808154,9.260000,170,1,1.000000,1,常驻,1常驻
4595,2022-02-13 22:47:00,2.808154,9.260000,170,1,1.000000,1,常驻,1常驻
4596,2022-02-13 22:48:00,2.932453,8.690770,170,1,1.000000,1,常驻,1常驻


In [19]:
df_join[3554:3566]

,time,x,y,label,zone_before,number,zone_after,category,zone_tag
3554,2022-02-10 13:57:00,3.5,5.0,111,3,1.0,3,常驻,3常驻
3555,2022-02-10 13:58:00,3.5,5.0,111,3,1.0,3,常驻,3常驻
3556,2022-02-10 13:59:00,3.5,5.0,111,3,1.0,3,常驻,3常驻
3557,2022-02-10 14:00:00,3.5,5.0,111,3,1.0,3,常驻,3常驻
3558,2022-02-10 14:01:00,3.5,5.0,111,3,1.0,3,常驻,3常驻
3559,2022-02-10 14:02:00,3.5,5.0,111,3,1.0,3,常驻,3常驻
3560,2022-02-10 14:03:00,3.5,5.0,111,3,1.0,3,常驻,3常驻
3561,2022-02-10 14:04:00,3.5,5.0,111,3,1.0,3,常驻,3常驻
3562,2022-02-10 14:05:00,3.5,5.0,111,3,1.0,3,常驻,3常驻
3563,2022-02-10 14:06:00,3.5,5.0,111,3,1.0,3,常驻,3常驻


## 新的时间轴处理函数

In [20]:
"""
Time line
"""
def createTimeLineAdvanced(key,details,res_path):
    timeline = []
    for zone in details:
        for t in details[zone]:
            timeline.append((t[0], t[1], zone))

    storeTimeLine(timeline,res_path+key+"_time_advanced.xlsx")

In [21]:
# 需要改 createTimeLine("All",details,res_path)需要更改"ALL"，区分长、中、短、全部
'''
获得{value=[time],key=所属分区}的字典
'''
dict_zones_all = df_join.groupby('zone_tag')['time'].apply(list).to_dict()
#print(dict_zones_all)
"""
get data_allInOne
"""

"""
get the data of corresponding zone
intervals: key=zone名称 value=最大连续停留时长
details: key=zone名称 value=所有停留时段 times_in_zone=[[start1,end1],[start2,end2],.....]
"""
intervals = {}
details = {}
for zone in dict_zones_all:
  intervals[zone],details[zone] = sliceByTime(dict_zones_all[zone],epsilon)

#print(details)
createTimeLineAdvanced("All",details,res_path)


In [22]:
#print(details['2.0常驻'])


In [23]:
#print(dict_zones_all['2.0常驻'])



# new_grids = createHeatmapByTime(dict_zones_all,"heatmap_allInOne",map_img,res_path,row_num,column_num,unit,max_x,min_x,max_y,min_y,groups)
# createHeatmapByTime2(new_grids,"heatmap_allInOne",map_img,res_path,unit,max_x,min_x,max_y,min_y)
